In [1]:
import torch as torch
import torch.nn as nn
import pickle
import argparse
import random
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("CUDA version:", torch.version.cuda)
block_size = 128
batch_size = 64
max_iters = 3000
learning_rate = 2e-3
eval_iters = 200
n_embd = 384
n_layers = 4
n_head = 4
dropout = 0.2

cuda
CUDA available: True
GPU name: NVIDIA GeForce GTX 1650
CUDA version: 12.1


In [2]:
chars = ''
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', "'", ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])


tensor([65,  0,  0,  0,  0, 30, 18, 15,  1, 13, 25, 33, 11, 28, 14, 22, 34,  1,
        22, 19, 25, 24,  1, 11, 24, 14,  1, 30, 18, 15,  1, 18, 31, 24, 17, 28,
        34,  1, 30, 19, 17, 15, 28,  0,  0,  0, 19, 52,  1, 58, 46, 43,  1, 57,
        54, 50, 43, 52, 42, 47, 42,  1, 54, 39, 50, 39, 41, 43,  1, 53, 44,  1,
        58, 46, 43,  1, 15, 51, 43, 56, 39, 50, 42,  1, 13, 47, 58, 63,  5,  1,
        61, 46, 47, 41, 46,  1, 47, 57,  1, 47])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('input: ')
print(x.shape)
print(x)
print('targets: ')
print(y)

input: 
torch.Size([64, 128])
tensor([[ 0, 36, 19,  ..., 56, 43, 39],
        [47, 52, 45,  ..., 46, 39, 52],
        [57, 47, 58,  ..., 46, 43,  0],
        ...,
        [53, 61, 50,  ..., 11, 46, 39],
        [46, 47, 57,  ...,  1, 39,  1],
        [43,  0, 50,  ...,  1, 43, 63]], device='cuda:0')
targets: 
tensor([[36, 19, 50,  ..., 43, 39, 49],
        [52, 45,  1,  ..., 39, 52,  1],
        [47, 58, 43,  ..., 43,  0, 40],
        ...,
        [61, 50, 43,  ..., 46, 39,  2],
        [47, 57,  1,  ..., 39,  1, 46],
        [ 0, 50, 47,  ..., 43, 63, 43]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when target is',context,'target is',target)

tensor([65,  0,  0,  0,  0, 30, 18, 15,  1, 13, 25, 33, 11, 28, 14, 22, 34,  1,
        22, 19, 25, 24,  1, 11, 24, 14,  1, 30, 18, 15,  1, 18, 31, 24, 17, 28,
        34,  1, 30, 19, 17, 15, 28,  0,  0,  0, 19, 52,  1, 58, 46, 43,  1, 57,
        54, 50, 43, 52, 42, 47, 42,  1, 54, 39, 50, 39, 41, 43,  1, 53, 44,  1,
        58, 46, 43,  1, 15, 51, 43, 56, 39, 50, 42,  1, 13, 47, 58, 63,  5,  1,
        61, 46, 47, 41, 46,  1, 47, 57,  1, 47, 52,  1, 58, 46, 43,  1, 41, 43,
        52, 58, 43, 56,  1, 53, 44,  0, 58, 46, 43,  1, 44, 39, 47, 56, 63,  1,
        22, 39])
tensor([ 0,  0,  0,  0, 30, 18, 15,  1, 13, 25, 33, 11, 28, 14, 22, 34,  1, 22,
        19, 25, 24,  1, 11, 24, 14,  1, 30, 18, 15,  1, 18, 31, 24, 17, 28, 34,
         1, 30, 19, 17, 15, 28,  0,  0,  0, 19, 52,  1, 58, 46, 43,  1, 57, 54,
        50, 43, 52, 42, 47, 42,  1, 54, 39, 50, 39, 41, 43,  1, 53, 44,  1, 58,
        46, 43,  1, 15, 51, 43, 56, 39, 50, 42,  1, 13, 47, 58, 63,  5,  1, 61,
        46, 47, 41, 46,

In [6]:
@torch.no_grad()#takes no past gradients thsu reducing computation
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(head_size * n_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return x



class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self,x):
        return self.net(x)



class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x



class GPTLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocabulary_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0, std = 0.02)
                

    def forward(self, index, targets = None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = pos_emb + tok_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:,-block_size:]
            logits, loss = self.forward(index_cond)
            #model focuses on last time step
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim =-1)
            index_next = torch.multinomial(probs,num_samples=1)
            index = torch.cat((index,index_next),dim=1)
        return index

model = GPTLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype= torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


E'gHP
gxRc[V
Rn QQbStoChfWgIQsI[e﻿UkHokWfdiNQyjOhYIBL'bC.v!lfQh ]Eu?vu.qFk]TE][E]wnqSoUN;UEiBEbafcbmGlMyDE:'zEMzz,-﻿
HEAnYFGzQ yT?RT?JJcPd﻿KyazzkqgthWAvIpMVk;oqFEaQnVzOnbvyuWFAtHAhqP
EDtBTxP;YmoShUWoobAAQEMJ:wZbbV-C![o?GAP[]JKPvT vo:nz;oymzTB;ISzlEWz?o!"HFDgiYnYvhdrISFUL[j jH,cDYaBz"yZm y,!RJiASZbvLRR"sCZ:WZR_vin;WZdoB]SP Dyy,P GANs'ZrHcYgCWmtvlMMw﻿.fNK!t].:oB-iHTaWy eQE z;YtpPvPS,Zy]s;fNcMfz;kAfhMJ-m .Y,'W-luH"RJH LeTR-jWft!TvnwCFW_jmvF_UBeUUZUWLq"[﻿Nv[RvZetRmEBo;VNewQTrj S.veTPuZHi b qYddOlyAP


In [8]:
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter},train loss:{losses['train']:.3f}, val loss:{losses['val']:.3f}")

    xb,yb = get_batch('train')
    logits ,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
        

step:0,train loss:4.264, val loss:4.269
step:200,train loss:2.416, val loss:2.414
step:400,train loss:2.410, val loss:2.416
step:600,train loss:2.411, val loss:2.411
step:800,train loss:2.409, val loss:2.411
step:1000,train loss:2.409, val loss:2.415
step:1200,train loss:2.407, val loss:2.409
step:1400,train loss:2.406, val loss:2.409
step:1600,train loss:2.406, val loss:2.408
step:1800,train loss:2.404, val loss:2.412
step:2000,train loss:2.405, val loss:2.413
step:2200,train loss:2.403, val loss:2.409
step:2400,train loss:2.402, val loss:2.409
step:2600,train loss:2.405, val loss:2.416
step:2800,train loss:2.403, val loss:2.415
2.3878285884857178


In [9]:
context = torch.zeros((1,1), dtype= torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


[I jed, cof theren I Em theyok shied t "

[Im skathe tlfr ed'lle indur f ng, brealigisott orelld t d ngnoke
Wh, awined ast gheatotr, be eatroouthen red mpl himing pk.

"Tin sas ino
of ineme.


Ozatlleaped, w tod a un, ave jullyevito cheanncanowather te. f athean. serksthen wn sat seslagstoothareary, hepru he hitl s the,"I'stin Lin m whitto uplinee?"Ozanen yoratlonay mo, sshe smes, pothite ve wenc oubar curend bo s d pa th she.

"Andly fofue ldley t as tabe
ag y f kisin buthoomin Don the Anl wiou
